In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import numpy as np


### DATA

class mfcc_set(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        data = self.X[index:index+1]
        # remove dimension 0
        data = data.squeeze(0)
        label = self.y[index]
        return torch.Tensor(data), torch.Tensor(label)

In [10]:
### MODEL
num_feature = 2
hidden_size = 4

class Model(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Model, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.rnn = nn.GRU(self.input_size, self.hidden_size, batch_first=True)
        self.fc = nn.Linear(self.hidden_size, self.output_size)
        
        
    # create function to init state
    def init_hidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size)
        
    
    def forward(self, x):     
        batch_size = x.size(0)
        h = self.init_hidden(batch_size)#.to(device)
        print(h.shape)
        out, h = self.rnn(x, h)       
        out = self.fc(out)
        
        #return out, h
        return out
        
    
model = Model(input_size=num_feature, hidden_size=hidden_size, output_size=1)
loss_func = nn.MSELoss()
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)



# model experiment

In [11]:
# num_feature = 64
# len_seq = 800
# batch_size = 
# h_size = (1, batch_size, hidden_size) --- (1,2,4)
# x_size = (batch_size, len_seq, num_feature) 
# y_size = (batch_size, len_seq, output_size=3)
batch_size = 2
x = torch.randn(batch_size, 1, num_feature)
ytest = model(
    x,
    # torch.zeros([1, batch_size, 4])
)
print(x)
print(x.shape)
print(ytest)
print(ytest.shape)


torch.Size([1, 2, 4])
tensor([[[-1.3263, -0.9296]],

        [[ 0.4254,  0.9082]]])
torch.Size([2, 1, 2])
tensor([[[-0.2312]],

        [[-0.0274]]], grad_fn=<AddBackward0>)
torch.Size([2, 1, 1])


In [ ]:
### TRAIN

num_epochs = 25
model.train()

for epoch in range(num_epochs):
    train_loss = 0
    
    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)
        
        yhat = model(x)
        loss = loss_func(yhat, y)
        #loss = loss_func(yhat[:, 1:, :], y[:, 1:, :])
        
        model.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss
    
    if not (epoch % 1):
        print(f'Epoch: {epoch+1:02d}, ' +
              f'Loss: {train_loss / len(train_dataloader.dataset):.4f}')

print('Finished Training')




### TEST

model.eval()

for test_seq_length in [train_seq_length, 1000, 4]:    
    test_dataset = Dataset(test_num_seq, test_seq_length, num_feature)
    test_dataloader = DataLoader(test_dataset, batch_size=2, shuffle=False)
    
    test_loss = 0
    
    for x, y in test_dataloader:
        x = x.to(device)
        y = y.to(device)

        with torch.no_grad():
            yhat = model(x)

        loss = loss_func(yhat, y)
        #loss = loss_func(yhat[:, 1:, :], y[:, 1:, :])
        test_loss += loss

    print(f'Average: Loss:{test_loss / len(test_dataloader.dataset):.4f}, seq_length:{test_seq_length}')

In [ ]:
import h5py
